<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
def load_dataset(filename):
#   df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
    df = pd.read_excel(filename, sheet_name="dataset_XY", encoding="utf8")
    Label_tag = pd.read_excel(filename, sheet_name="Label_tag", encoding="utf8")  
    print(df.head())
    intent = df['Label']
    for idx, i in enumerate(intent):
#         print(i)
        intent[idx] = Label_tag.loc[i,"Tag"]
#     print("intent: ", intent)
    unique_intent = list(set(intent))
    sentences = list(df["Questions"])
  
    return (intent, unique_intent, sentences)


In [3]:
intent, unique_intent, sentences = load_dataset("dataset_XY_XLS_updatedbytho.xls")
print("Unique intents: ", unique_intent)
print("Number of intents: ", len(unique_intent))
# print(intent[:5])
# len(intent)

           Questions  Label
0               Chào      0
1    Dạo này thế nào      0
2  Có ai ở đây không      0
3           Xin chào      0
4           Xao chìn      0
Unique intents:  ['Tạm_biệt', 'Chuẩn_bị_hồ_sơ', 'Cảm_ơn', 'Thời_gian_trả_kết_quả', 'Chức_năng_của_bot', 'Tên_bot', 'Nơi_nộp_hồ_sơ', 'Nơi_nhận_hộ_chiếu', 'Chào_hỏi', 'Thời_gian_giải_quyết_hộ_chiếu', 'Thời_gian_tiếp_nhận_hồ_sơ', 'Lệ_phí', 'Trình_tự ', 'Đối_tượng_xin_cấp_hộ_chiếu', 'Các_loại_hộ_chiếu']
Number of intents:  15


C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
# Hàm được viết bởi thọ
import pyvi.ViTokenizer as viToken
def viTokenList(word):
    return viToken.tokenize(word).split()
viTokenList("Anh thọ quá đẹp trai!")

['Anh', 'thọ', 'quá', 'đẹp_trai', '!']

In [5]:
print(sentences[:5])

['Chào', 'Dạo này thế nào', 'Có ai ở đây không', 'Xin chào', 'Xao chìn']


In [6]:
# nltk.download("stopwords")
# nltk.download("punkt")

In [7]:
#define stemmer
# stemmer = LancasterStemmer()

In [8]:
def cleaning(sentences):
    words = []
    for s in sentences:
#         clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s) # Find and replace duplicate 
        w = viTokenList(s)
        #stemming
        words.append([i.lower() for i in w])

    return words  

In [9]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


750
[['chào'], ['dạo', 'này', 'thế_nào']]


In [10]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

In [11]:
def max_length(words):
    return(len(max(words, key = len)))
  

In [12]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 318 and Maximum length = 16


In [13]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [14]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [15]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [16]:
padded_doc = padding_doc(encoded_doc, max_length)

In [17]:
padded_doc[:5]

array([[115,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [180,  99,  21,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  5,  55,  24, 126,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 17, 115,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [204,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [18]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (750, 16)


In [19]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [20]:
output_tokenizer.word_index

{'tạm_biệt': 1,
 'chuẩn_bị_hồ_sơ': 2,
 'cảm_ơn': 3,
 'thời_gian_trả_kết_quả': 4,
 'chức_năng_của_bot': 5,
 'tên_bot': 6,
 'nơi_nộp_hồ_sơ': 7,
 'nơi_nhận_hộ_chiếu': 8,
 'chào_hỏi': 9,
 'thời_gian_giải_quyết_hộ_chiếu': 10,
 'thời_gian_tiếp_nhận_hồ_sơ': 11,
 'lệ_phí': 12,
 'trình_tự': 13,
 'đối_tượng_xin_cấp_hộ_chiếu': 14,
 'các_loại_hộ_chiếu': 15}

In [21]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [22]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [23]:
encoded_output.shape

(750, 1)

In [24]:
def one_hot(encode):
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

In [25]:
output_one_hot = one_hot(encoded_output)

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [26]:
output_one_hot.shape

(750, 15)

In [27]:
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [28]:
# print(len(padded_doc))
# print(len(output_one_hot))
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [29]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (600, 16) and train_Y = (600, 15)
Shape of val_X = (150, 16) and val_Y = (150, 15)


In [30]:
def create_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
    model.add(Bidirectional(LSTM(128)))
    #   model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(15, activation = "softmax"))

    return model

In [31]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 128)           40704     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                495       
Total params: 312,591
Trainable params: 271,887
Non-trainable params: 40,704
_________________________________________________________________


In [32]:
# filename = 'model.h5'
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# hist = model.fit(train_X, train_Y, epochs = 300, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

In [33]:
 model = load_model("model.h5")

In [34]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True,show_layer_names=True)
# from IPython.display import display, Image
# display(Image(filename='model.png'))


In [35]:
import matplotlib.pyplot as plt

# # Plot training & validation accuracy values
# plt.plot(hist.history['acc'])
# plt.plot(hist.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [36]:
def predictions(text):
#     clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = viTokenList(text)
    test_word = [w.lower() for w in test_word]
    test_ls = word_tokenizer.texts_to_sequences(test_word)
#     print("test_ls: ", test_ls)
#     print("test_word: ", test_word)
    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))

    test_ls = np.array(test_ls).reshape(1, len(test_ls))

    x = padding_doc(test_ls, max_length)

    pred = model.predict_proba(x)


    return pred


  

In [37]:
def load_response_dataset(filename):
    df_res = pd.read_excel(filename, sheet_name="Tag_Response", encoding="utf8") 
    return df_res

In [38]:
import random
def response(classes):
    df_res = load_response_dataset("dataset_XY_XLS_updatedbytho.xls")
    s = df_res[classes].dropna()
    s_res = s[random.randint(0,len(s)-1)]
    print(s_res)

In [39]:
def get_final_output(pred, classes):
    predictions = pred[0]

    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
    predictions = -np.sort(-predictions)
 
    for i in range(3):
        print("%s has confidence = %s" % (classes[i], (predictions[i])))
    response(classes[0])
#     print(classes[0])

In [ ]:

while(1):
    text = input()    
    if text == 'q':
        break
    pred = predictions(text)
    get_final_output(pred, unique_intent)
    

hi
Chức_năng_của_bot has confidence = 0.7883284
Nơi_nộp_hồ_sơ has confidence = 0.14198619
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.039299227
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương và cấp tỉnh. Bạn nên lưu ý ghi rõ là cấp nào.
ngu luôn
Chức_năng_của_bot has confidence = 0.55176234
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.34406018
Tên_bot has confidence = 0.07534421
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương và cấp tỉnh. Bạn nên lưu ý ghi rõ là cấp nào.
sao m bị ngu rồi?
Chức_năng_của_bot has confidence = 0.54097444
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.3523135
Tên_bot has confidence = 0.07309757
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương 